In [ ]:
import sys; sys.path.append('../3rdparty/ElasticRods/python')

import elastic_rods, elastic_knots
from linkage_vis import LinkageViewer as Viewer
import numpy as np, matplotlib.pyplot as plt, matplotlib, os, json

from helpers import *

%load_ext autoreload
%autoreload 2

In [ ]:
# Download the sample dataset (equilibrium states of knots with up to 9 crossings)
# The same data can also be downloaded manually from 
# https://drive.google.com/file/d/1mwGCbWD8-Ftku5eLRwOAuvM-NGCqdXfg/
root_dir = '../data/'
data_dir = 'L400-r0.2-UpTo9Crossings'

zip_file = data_dir + '.zip'
download_data(
    gdrive_id='1mwGCbWD8-Ftku5eLRwOAuvM-NGCqdXfg',
    output_dir=root_dir,
    zip_file=zip_file
)

# Analysis of the simulation results

In [ ]:
# Load the KnotInfo knot table (see https://knotinfo.math.indiana.edu for more details)
knot_data = load_knot_table()
knot_data.head(3)

In [ ]:
# Load the attributes of the computed equilibrium states
import pandas as pd

eq_data = pd.DataFrame(columns=[
    'knot_type',
    'eq_id',
    'energy',
    'energy_bend',
    'energy_twist',
    'energy_stretch',
    'max_bend_stress',
    'total_curvature',
    'threedim',
    'spher_dev',
    'diameter',
    'writhe',
    'twist',
    'length_pc0',
    'length_pc1',
    'length_pc2',
    'lambda_7',
    'c_hull_area',
    'c_hull_volume',
])

data_path = root_dir + '{}/'.format(data_dir)
knot_types = [kt for kt in sorted_nicely(os.listdir(data_path)) if not kt == 'README.md']
for kt in knot_types:
    stats_path = data_path + '{}/'.format(kt) + 'stats.json'
    
    eq_data_curr_knot = pd.read_json(stats_path, dtype='float')
    eq_data_curr_knot.insert(0, 'eq_id', np.arange(eq_data_curr_knot.shape[0]))  # serial corrseponding to the .obj file
    eq_data_curr_knot.insert(0, 'knot_type', kt)
    
    eq_data = pd.concat([eq_data, eq_data_curr_knot], ignore_index=True)
    
eq_data[eq_data.columns[1:]] = eq_data[eq_data.columns[1:]].apply(pd.to_numeric)  # cast values to float
print("Attributes of {} equilibrium states have been loaded".format(eq_data.shape[0]))

In [ ]:
eq_data.head(3)

In [ ]:
eq_data.tail(3)

### Filter and sort equilibria by attribute

In [ ]:
def sort_knot_types_by_attribute(df, attribute, minimum=True, n_max=9):
    "Get the equilibrium states with max/min value of the given attribute (select at most one state per knot type)."
    
    if minimum:
        return df.loc[df.groupby('knot_type')[attribute].idxmin()].sort_values(by=attribute, ascending=minimum).iloc[0:n_max]
    else:
        return df.loc[df.groupby('knot_type')[attribute].idxmax()].sort_values(by=attribute, ascending=minimum).iloc[0:n_max]

In [ ]:
rod_radius = 0.2
material = elastic_rods.RodMaterial('ellipse', 2000, 0.3, [rod_radius, rod_radius])

In [ ]:
# Equilibria with the lowest energy
df = sort_knot_types_by_attribute(eq_data, 'energy', minimum=True)
rods = load_rods_from_dataframe(df, data_path=data_path, material=material)

view_energy = Viewer(rods, width=1024, height=640)
view_energy.show()

In [ ]:
# Equilibria with the largest three-dimensionality score
df = sort_knot_types_by_attribute(eq_data, 'threedim', minimum=False, n_max=1)
rods = load_rods_from_dataframe(df, data_path=data_path, material=material)

view_threedim_high = Viewer(rods, width=1024, height=640)
view_threedim_high.show()

In [ ]:
# Equilibria with the smallest three-dimensionality score
df = sort_knot_types_by_attribute(eq_data, 'threedim', minimum=True)
rods = load_rods_from_dataframe(df, data_path=data_path, material=material)

view_threedim_low = Viewer(rods, width=1024, height=640)
view_threedim_low.show()

### Milnor's bound
Check that, for any non-trivial knot, 
$$
\int_0^L \kappa > 2 \pi \text{b},
$$

where $\kappa = \Vert \mathbf{x}'' \Vert$ is the local curvature of the centerline curve $\mathbf{x}$, and $\text{b}$ is the bridge index of the corresponding knot type (see [[Milnor 1950]](https://www.jstor.org/stable/1969467)).

In case the bridge index of the knot type coincides with its bridge index (BB knots, see [[Diao et al. 2021]](https://arxiv.org/abs/2108.11790)), the global energy minimizer is known to be a multi-covered circle.

In [ ]:
# Fetch bridge and braid indices from the knot table and check if knot type is BB
eq_data_milnor = eq_data.merge(
    knot_data[['Name', 'Bridge Index', 'Braid Index']], 
    left_on='knot_type', right_on='Name'
).drop('Name', axis=1)
eq_data_milnor = eq_data_milnor.rename(columns={'Bridge Index': 'bridge_index','Braid Index': 'braid_index'})
eq_data_milnor['is_bb'] = (eq_data_milnor['bridge_index'] == eq_data_milnor['braid_index'])

# Compute the "Milnor's distance",
# i.e. the ratio between the integrated curvature 
# and 2pi times the bridge index: it should always be > 1
eq_data_milnor['milnor_dist'] = eq_data_milnor['total_curvature'] / (2*np.pi*eq_data_milnor['bridge_index'])

In [ ]:
# For each knot type, selected the equilibrium state with the lowest total curvature
df_min_tot_curv = eq_data_milnor.loc[eq_data_milnor.groupby('knot_type').total_curvature.idxmin()]
md = df_min_tot_curv.milnor_dist.values
md_bb = df_min_tot_curv.milnor_dist[df_min_tot_curv.is_bb].values
md_not_bb = df_min_tot_curv.milnor_dist[~df_min_tot_curv.is_bb].values
n_bb = md_bb.size
n_not_bb = md_not_bb.size

# Plot histogram of Milnor's distances
fig, ax = plt.subplots()
cmap = matplotlib.cm.get_cmap('Blues')
blue_dark, blue, blue_light = cmap(0.9), cmap(0.6), cmap(0.2)

fig_dummy, ax_dummy = plt.subplots()
nbins = 40
hist_not_bb = ax_dummy.hist(md_not_bb, bins=np.linspace(np.min(md_not_bb), np.max(md_not_bb), nbins))
bins_hist_not_bb = hist_not_bb[1]
plt.close()

md_not_bb_bin1 = md_not_bb[md_not_bb <= bins_hist_not_bb[2]]
md_not_bb_bin2 = md_not_bb[md_not_bb > bins_hist_not_bb[-2]]
ax.hist(md_not_bb, bins=np.linspace(np.min(md_not_bb), np.max(md_not_bb), nbins), color=blue, label='Non-BB knot types ({})'.format(n_not_bb))
ax.hist(md_not_bb_bin2, bins=bins_hist_not_bb[-2:], color=blue_dark)
ax.hist(md_not_bb_bin1, bins=bins_hist_not_bb[0:2], color=blue_light)
ax.hist(md_bb, bins=[1, np.sort(md_bb)[-1]+1e-8, bins_hist_not_bb[0], bins_hist_not_bb[1]], color='C1', label='BB knot types ({})'.format(n_bb))
ax.legend()
ax.set_xlabel('$\int_0^L \kappa / 2 \pi b$')
ax.set_ylabel('Knot types count')
ax.set_title('Milnor\'s Distance Distribution')
plt.show()

In [ ]:
# Select the knots to display from the dataset
df_bb = df_min_tot_curv[df_min_tot_curv.is_bb]
df_not_bb_1 = df_min_tot_curv[(~df_min_tot_curv.is_bb) & (df_min_tot_curv.milnor_dist < bins_hist_not_bb[ 1])].sort_values(by='milnor_dist')  # light blue
df_not_bb_2 = df_min_tot_curv[(~df_min_tot_curv.is_bb) & (df_min_tot_curv.milnor_dist > bins_hist_not_bb[-2])].sort_values(by='milnor_dist')  # dark blue

# Load the corresponding elastic rods
rod_radius = 0.2
material = elastic_rods.RodMaterial('ellipse', 2000, 0.3, [rod_radius, rod_radius])
rods_bb = load_rods_from_dataframe(df_bb, data_path=data_path, material=material)
rods_not_bb_1 = load_rods_from_dataframe(df_not_bb_1, data_path=data_path, material=material)
rods_not_bb_2 = load_rods_from_dataframe(df_not_bb_2, data_path=data_path, material=material)

In [ ]:
# BB knots
print(df_bb[['knot_type', 'milnor_dist']].to_string(index=False))
view_bb = Viewer(rods_bb, width=1024, height=640)
view_bb.show()

In [ ]:
# Non-BB knots with the smallest Milnor's distance
print(df_not_bb_1[['knot_type', 'milnor_dist']].to_string(index=False))
view_not_bb_1 = Viewer(rods_not_bb_1, width=1024, height=640)
view_not_bb_1.show()

In [ ]:
# Non-BB knots with the largest Milnor's distance
print(df_not_bb_2[['knot_type', 'milnor_dist']].to_string(index=False))
view_not_bb_2 = Viewer(rods_not_bb_2, width=1024, height=640)
view_not_bb_2.show()